In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import sqlite3
from functools import partial

class QuizApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Quiz App")

         # Set the icon
        #self.root.iconbitmap("brain.ico")
        
        # Database setup
        self.conn = sqlite3.connect("user_data.db")
        self.c = self.conn.cursor()
        self.c.execute("CREATE TABLE IF NOT EXISTS users (username TEXT, password TEXT, score INTEGER)")
        self.conn.commit()
        
        # Menu Bar
        self.menu_bar = tk.Menu(root)
        root.config(menu=self.menu_bar)

        self.file_menu = tk.Menu(self.menu_bar, tearoff=0)
        self.menu_bar.add_cascade(label="File", menu=self.file_menu)

        # User Entry Fields
        self.username_label = ttk.Label(root, text="Username:")
        self.username_label.grid(row=0, column=0, padx=10, pady=10, sticky=tk.E)

        self.username_entry = ttk.Entry(root)
        self.username_entry.grid(row=0, column=1, padx=10, pady=10, sticky=tk.W)

        self.password_label = ttk.Label(root, text="Password:")
        self.password_label.grid(row=1, column=0, padx=10, pady=10, sticky=tk.E)

        self.password_entry = ttk.Entry(root, show="*")
        self.password_entry.grid(row=1, column=1, padx=10, pady=10, sticky=tk.W)

        # Submit Button
        self.login_button = ttk.Button(root, text="Login", command=self.login)
        self.login_button.grid(row=2, column=0, columnspan=2, pady=10)

         # Create Account Button
        self.create_account_button = ttk.Button(root, text="Create Account", command=self.create_account)
        self.create_account_button.grid(row=3, column=0, columnspan=2, pady=10)

        # Quiz Frame (hidden initially)
        self.quiz_frame = ttk.Frame(root)
        
        # Quiz data
        self.quizzes = [
            {"name": "General Knowledge", "questions": [
                {"question": "Is Paris the capital of France?", "answer": "yes"},
                {"question": "Is Mars known as the Red Planet?", "answer": "yes"},
                # Add more questions as needed
            ]},
            {"name": "Science Quiz", "questions": [
                {"question": "Is water composed of two hydrogen atoms and one oxygen atom?", "answer": "yes"},
                {"question": "Is the Earth flat?", "answer": "no"},
                # Add more questions as needed
            ]},
            # Add more quizzes as needed
        ]
        self.current_quiz_index = 0
        self.score = 0

        # Continue Button
        self.continue_button = ttk.Button(self.quiz_frame, text="Continue to next quiz", command=self.next_question)
        self.continue_button.pack(pady=10)

        # Leaderboard Button
        self.leaderboard_button = ttk.Button(self.quiz_frame, text="Leaderboard", command=self.show_leaderboard)
        self.leaderboard_button.pack(pady=10)

        # Logout Button
        self.logout_button = ttk.Button(self.quiz_frame, text="Logout", command=self.logout)
        self.logout_button.pack(pady=10)

        # Review Frame (hidden initially)
        self.review_frame = ttk.Frame(root)
        self.review_text = tk.Text(self.review_frame, wrap=tk.WORD, width=40, height=10)
        self.review_text.pack(pady=10)

        # Review Back Button
        self.review_back_button = ttk.Button(self.review_frame, text="Back to Quiz", command=self.hide_review)
        self.review_back_button.pack(pady=10)

        # Timer label
        self.timer_label = ttk.Label(self.quiz_frame, text="Timer:")
        self.timer_label.pack(pady=10)

        # Timer variables
        self.timer_seconds = 10
        self.timer_running = False

        # Progress Label
        self.progress_label = ttk.Label(self.quiz_frame, text="")
        self.progress_label.pack(pady=10)


        # Save initial UI state for reset
        self.initial_ui_state = {
            "username_label": self.username_label.cget("text"),
            "password_label": self.password_label.cget("text"),
            "login_button": self.login_button.cget("text"),
            "create_account_button": self.create_account_button.cget("text"),
        }

    def show_login_frame(self):
        self.quiz_frame.pack_forget()
        self.username_entry.delete(0, tk.END)
        self.password_entry.delete(0, tk.END)
        for key, value in self.initial_ui_state.items():
            getattr(self, key).config(text=value)

    def logout(self):
        self.show_login_frame()

    def login(self):
        username = self.username_entry.get()
        password = self.password_entry.get()

        # Authenticate user (you might want to improve this part)
        self.c.execute("SELECT * FROM users WHERE username=? AND password=?", (username, password))
        user = self.c.fetchone()

        if user:
            self.show_quiz_frame()
        else:
            messagebox.showerror("Error", "Invalid login credentials")

    # def show_quiz_frame(self):
    #     self.quiz_frame.pack()
    #     self.current_quiz_index = 0
    #     self.score = 0
    #     self.next_question()

    def show_quiz_frame(self):
        # Remove all children managed by grid
        for child in self.root.grid_slaves():
            child.grid_forget()
        
        # Now you can use pack on self.quiz_frame
        self.quiz_frame.pack()
        self.current_quiz_index = 0
        self.score = 0
        self.next_question()


    def create_account(self):
        create_account_window = tk.Toplevel(self.root)
        create_account_window.title("Create Account")

        # User Entry Fields
        username_label = ttk.Label(create_account_window, text="Username:")
        username_label.grid(row=0, column=0, padx=10, pady=10, sticky=tk.E)

        username_entry = ttk.Entry(create_account_window)
        username_entry.grid(row=0, column=1, padx=10, pady=10, sticky=tk.W)

        password_label = ttk.Label(create_account_window, text="Password:")
        password_label.grid(row=1, column=0, padx=10, pady=10, sticky=tk.E)

        password_entry = ttk.Entry(create_account_window, show="*")
        password_entry.grid(row=1, column=1, padx=10, pady=10, sticky=tk.W)

        # Create Account Button
        create_account_button = ttk.Button(create_account_window, text="Create Account", command=partial(self.save_account, username_entry, password_entry))
        create_account_button.grid(row=2, column=0, columnspan=2, pady=10)

    def save_account(self, username_entry, password_entry):
        username = username_entry.get()
        password = password_entry.get()

        # Check if the username already exists
        self.c.execute("SELECT * FROM users WHERE username=?", (username,))
        existing_user = self.c.fetchone()

        if existing_user:
            messagebox.showerror("Error", "Username already exists. Please choose another.")
        else:
            self.c.execute("INSERT INTO users (username, password, score) VALUES (?, ?, 0)", (username, password))
            self.conn.commit()
            messagebox.showinfo("Success", "Account created successfully!")
        


    def next_question(self):
        if not self.timer_running:
            self.start_timer()
        if self.current_quiz_index < len(self.quizzes):
            quiz_data = self.quizzes[self.current_quiz_index]
            quiz_name = quiz_data["name"]

            if quiz_data["questions"]:
                question_data = quiz_data["questions"].pop(0)
                question = question_data["question"]
                correct_answer = question_data["answer"]

                # Display question
                user_answer = messagebox.askquestion(quiz_name, f"{question}\nYes or No")

                # Record user's answer for review
                self.review_text.insert(tk.END, f"Q: {question}\nYour Answer: {user_answer.lower()}\nCorrect Answer: {correct_answer}\n\n")

                if user_answer.lower() == correct_answer:
                    self.score += 1

                # Update progress label
                progress_text = f"Current Progress: Quiz {self.current_quiz_index + 1}/{len(self.quizzes[self.current_quiz_index]['questions'])} - Current Score: {self.score}"
                self.progress_label.config(text=progress_text)

                # Check if there are more questions in the quiz
                if not quiz_data["questions"]:
                    self.current_quiz_index += 1
            else:
                self.current_quiz_index += 1

            self.continue_button.invoke()  # Automatically go to the next question or quiz
        else:
            self.show_results()
    
    def start_timer(self):
        self.timer_running = True
        self.update_timer()

    def update_timer(self):
        if self.timer_running:
            self.timer_label.config(text=f"Timer: {self.timer_seconds}")
            if self.timer_seconds > 0:
                self.timer_seconds -= 1
                self.root.after(1000, self.update_timer)
            else:
                self.timer_running = False
                self.handle_timeout()

    def handle_timeout(self):
        timeout_message = "Time's up! Moving to the next question."
        messagebox.showinfo("Timeout", timeout_message)
    
        # Schedule the closing of the popup after 2 seconds
        self.root.after(2000, lambda: self.close_timeout_popup())

    def close_timeout_popup(self):
        # Close the popup
        self.root.after(1, lambda: self.root.focus_force())

    def show_results(self):
        messagebox.showinfo("Results", f"Your score: {self.score}/{len(self.quizzes[0]['questions'])}")
        self.save_score()
        self.continue_button.config(state=tk.DISABLED)

    def save_score(self):
        username = self.username_entry.get()
        self.c.execute("UPDATE users SET score=? WHERE username=?", (self.score, username))
        self.conn.commit()
        self.timer_seconds = 10
        self.timer_running = False

    def show_leaderboard(self):
        self.c.execute("SELECT username, score FROM users ORDER BY score DESC")
        leaderboard = self.c.fetchall()
        leaderboard_text = "\n".join([f"{user[0]}: {user[1]}" for user in leaderboard])
        messagebox.showinfo("Leaderboard", f"Leaderboard:\n{leaderboard_text}")

    def show_review(self):
        self.review_frame.pack()

    def hide_review(self):
        self.review_frame.pack_forget()
        self.quiz_frame.pack_forget()
        self.continue_button.config(state=tk.NORMAL)

    def logout(self):
        self.quiz_frame.pack_forget()
        self.continue_button.config(state=tk.NORMAL)

# Run the app
root = tk.Tk()
app = QuizApp(root)
root.mainloop()
